In [1]:
import sys
sys.argv=['']
del sys
import os 
import keras
import keras.backend as K
import glob
import shutil
from PIL import Image, ImageOps
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from base_model import *
from config import *
from utils import SnapshotCallbackBuilder, bce_dice_loss, dice_coef_3cat_loss, my_iou_metric, my_bg_metric
from generators2 import CombinedDataGenerator, DataGenerator, AUGMENTATIONS_TRAIN, AUGMENTATIONS_TEST
from predict import *

C:\Users\Tomer\AppData\Roaming\Python\Python38\site-packages\albumentations\augmentations\transforms.py:1826: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
C:\Users\Tomer\AppData\Roaming\Python\Python38\site-packages\albumentations\augmentations\transforms.py:1800: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [3]:
#if __name__ == '__main__':

if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)

train_im_path, train_mask_path = os.path.join(args.train_path,'Orthophoto'), os.path.join(args.train_path,'Ground_truth')
h, w, batch_size = args.image_hight, args.image_width, args.batch_size

training_generator = DataGenerator(batch_size=2, augmentations=AUGMENTATIONS_TRAIN,img_size=512, train_im_path=train_im_path,train_mask_path=train_mask_path)
validation_generator = DataGenerator(batch_size=2, augmentations=AUGMENTATIONS_TEST,img_size=512, train_im_path=train_im_path,train_mask_path=train_mask_path) #TODO change the paths at runtime.

snapshot = SnapshotCallbackBuilder(nb_epochs=args.epochs,nb_snapshots=1,init_lr=1e-3)

model = build_effienet_unet(input_shape=(128, 128, 3))
model.summary()

model.compile(loss=dice_coef_3cat_loss, optimizer='adam', metrics=[my_iou_metric, my_bg_metric])
model.load_weights('Model_Test\model_weights.h5')


Model: "EfficientNetB7_UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 513, 513, 3)  0           ['normalization

                                                                                                  
 block1c_bn (BatchNormalization  (None, 256, 256, 32  128        ['block1c_dwconv[0][0]']         
 )                              )                                                                 
                                                                                                  
 block1c_activation (Activation  (None, 256, 256, 32  0          ['block1c_bn[0][0]']             
 )                              )                                                                 
                                                                                                  
 block1c_se_squeeze (GlobalAver  (None, 32)          0           ['block1c_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block1c_s

 )                              2)                                                                
                                                                                                  
 block2a_se_squeeze (GlobalAver  (None, 192)         0           ['block2a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block2a_se_reshape (Reshape)   (None, 1, 1, 192)    0           ['block2a_se_squeeze[0][0]']     
                                                                                                  
 block2a_se_reduce (Conv2D)     (None, 1, 1, 8)      1544        ['block2a_se_reshape[0][0]']     
                                                                                                  
 block2a_se_expand (Conv2D)     (None, 1, 1, 192)    1728        ['block2a_se_reduce[0][0]']      
          

 block2c_se_reshape (Reshape)   (None, 1, 1, 288)    0           ['block2c_se_squeeze[0][0]']     
                                                                                                  
 block2c_se_reduce (Conv2D)     (None, 1, 1, 12)     3468        ['block2c_se_reshape[0][0]']     
                                                                                                  
 block2c_se_expand (Conv2D)     (None, 1, 1, 288)    3744        ['block2c_se_reduce[0][0]']      
                                                                                                  
 block2c_se_excite (Multiply)   (None, 128, 128, 28  0           ['block2c_activation[0][0]',     
                                8)                                'block2c_se_expand[0][0]']      
                                                                                                  
 block2c_project_conv (Conv2D)  (None, 128, 128, 48  13824       ['block2c_se_excite[0][0]']      
          

                                                                                                  
 block2e_se_reshape (Reshape)   (None, 1, 1, 288)    0           ['block2e_se_squeeze[0][0]']     
                                                                                                  
 block2e_se_reduce (Conv2D)     (None, 1, 1, 12)     3468        ['block2e_se_reshape[0][0]']     
                                                                                                  
 block2e_se_expand (Conv2D)     (None, 1, 1, 288)    3744        ['block2e_se_reduce[0][0]']      
                                                                                                  
 block2e_se_excite (Multiply)   (None, 128, 128, 28  0           ['block2e_activation[0][0]',     
                                8)                                'block2e_se_expand[0][0]']      
                                                                                                  
 block2e_p

...

 lization)                                                                                        
                                                                                                  
 block5d_drop (Dropout)         (None, 32, 32, 224)  0           ['block5d_project_bn[0][0]']     
                                                                                                  
 block5d_add (Add)              (None, 32, 32, 224)  0           ['block5d_drop[0][0]',           
                                                                  'block5c_add[0][0]']            
                                                                                                  
 block5e_expand_conv (Conv2D)   (None, 32, 32, 1344  301056      ['block5d_add[0][0]']            
                                )                                                                 
                                                                                                  
 block5e_e

                                                                                                  
 block5f_add (Add)              (None, 32, 32, 224)  0           ['block5f_drop[0][0]',           
                                                                  'block5e_add[0][0]']            
                                                                                                  
 block5g_expand_conv (Conv2D)   (None, 32, 32, 1344  301056      ['block5f_add[0][0]']            
                                )                                                                 
                                                                                                  
 block5g_expand_bn (BatchNormal  (None, 32, 32, 1344  5376       ['block5g_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block5g_e

                                                                                                  
 block5i_expand_conv (Conv2D)   (None, 32, 32, 1344  301056      ['block5h_add[0][0]']            
                                )                                                                 
                                                                                                  
 block5i_expand_bn (BatchNormal  (None, 32, 32, 1344  5376       ['block5i_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block5i_expand_activation (Act  (None, 32, 32, 1344  0          ['block5i_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 block5i_d

                                                                                                  
 block6a_expand_bn (BatchNormal  (None, 32, 32, 1344  5376       ['block6a_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block6a_expand_activation (Act  (None, 32, 32, 1344  0          ['block6a_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 64, 64, 512)  2753024    ['block6a_expand_activation[0][0]
 ose)                                                            ']                               
                                                                                                  
 concatena

                                                                                                  
 conv2d_6 (Conv2D)              (None, 512, 512, 64  38656       ['concatenate_3[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 512, 512, 64  256        ['conv2d_6[0][0]']               
 rmalization)                   )                                                                 
                                                                                                  
 activation_6 (Activation)      (None, 512, 512, 64  0           ['batch_normalization_6[0][0]']  
                                )                                                                 
                                                                                                  
 conv2d_7 

In [ ]:
# history = model.fit(training_generator,
#                 validation_data=training_generator,                            
#                 use_multiprocessing=False,
#                 epochs=args.epochs,verbose=1,
#                 callbacks=snapshot.get_callbacks())

In [ ]:
img_dir = train_im_path+"\ortho_100.png"
image = predict_full_image(model, img_dir)
image.show(0)

In [ ]:
# model_a = predict_image(model, image)
# model_b = predict_image(model, image)
# pred_images = [model_a, model_b]
# ensemble_pred = ensemble(pred_images)